In [1]:
from model_utils import OpenImagesDataset, IMAGE_SIZE, imageTransforms
from torch.utils.data import DataLoader
import torch

In [2]:
# Creating the dataset
trainingData = OpenImagesDataset(rootDirectory='open-images-v6', 
                                 anchorBoxes='centroids.npy', 
                                 transform=imageTransforms, 
                                 dataType='train')     

In [3]:
# Defining the dataloader
trainDataLoader = DataLoader(dataset=trainingData, 
                             batch_size=32,
                             num_workers=8)

In [4]:
# Computing the total number of pixels (its the same for each channel)
numPixels = len(trainingData) * IMAGE_SIZE * IMAGE_SIZE

In [5]:
# Defining tensor to hold the totals for each channel
total = torch.tensor([0.,0.,0.])

# Iterating through each bach, and summing up the values in each channgel respectively
for batch_ndx, (data, label) in enumerate(trainDataLoader):
    total += torch.sum(data, dim=[0,2,3])

# Computing the mean
mean = total / numPixels

In [6]:
# Defining tensor to hold the totals for each channel
total = torch.tensor([0.,0.,0.])

# Iterating through each bach, and summing up the values in each channgel respectively
for batch_ndx, (data, label) in enumerate(trainDataLoader):
    # Data Shape: BatchSize/NumChannels/ImageSize/ImageSize -> BatchSize/NumChannels/ImageSize*ImageSize
    data = data.view((data.shape[0], 3, -1))
    
    # Data Shape: BatchSize/NumChannels/ImageSize*ImageSize -> BatchSize/ImageSize*ImageSize/NumChannels
    data = data.transpose(1,2)
    
    # Computing the inside of the sqrt
    total += torch.sum((data-mean)**2, dim=[0,1])

# Computing the STD
std = torch.sqrt(total/numPixels) 

In [7]:
print('Channel Means: ',mean)

Channel Means:  tensor([0.4777, 0.4482, 0.3984])


In [8]:
print('Channel STDs: ',std)

Channel STDs:  tensor([0.2717, 0.2655, 0.2715])
